# estimate_ARparams_by_NNで推定した$F$を用いてwithoutDiscriminatorで$G$を学習

In [1]:
import argparse
import os
path = os.getcwd()
path=path[:path.find('estimate-innovation')+20]
# No = (os.path.basename(__file__))[-4]
No = "0" # notebook用
# print('実験No.'+No)
import warnings
warnings.simplefilter('ignore')# 警告を非表示
import numpy as np
np.set_printoptions(precision=4, floatmode='fixed', suppress=True)
import matplotlib.pyplot as plt
from scipy import stats
import math
import sys
sys.path.append(path)
import random
import time
import statsmodels.api as sm
from scipy.stats import norm
import japanize_matplotlib
from scipy.stats import gaussian_kde

import torchvision.transforms as transforms
from torchvision.utils import save_image

import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable
# 人工データを生成してくれる機械が置いてあるところ
import tsModel
# 学習用のニューラルネットが置いてあるところ
import models
# p-Wasserstein距離の関数
import Wasserstein
path

'/home/lab/ryuichi/Documents/research/estimate-innovation/'

### ARモデルに従う時系列の作成（Fの学習と同じやつを使う）

In [2]:
phi_a = [1.0, -0.5, 0.7, -0.4]
phi_b = [0]
p = len(phi_a)
q = len(phi_b)
dataSeed=5
N = 1000
data = tsModel.ARIMA(a=phi_a, b=phi_b, N=N, random_seed=dataSeed, randomness='normal')
inno = tsModel.ARIMA(a=phi_a, b=phi_b, N=N, random_seed=dataSeed, randomness='normal', return_innovation=True)

### GPUが使えるかどうかを確認

In [3]:
import checkGPU
cuda, device = checkGPU.checkGPU()

GPUが使えます。
GPUを使いますか？ （Yes：1, No：0）： 0


### 推定に用いる$\hat p$を決定

In [4]:
hat_p = 4

### 学習曲線などの画像を保管するディレクトリと、NNのパラメータを保存するディレクトリを作成

In [5]:
os.makedirs("output-images/p{0}".format(hat_p), exist_ok=True)
os.makedirs("parameters/p{0}".format(hat_p), exist_ok=True)

### NNを学習する時のハイパラを決定する

In [6]:
parser = argparse.ArgumentParser()
parser.add_argument("--generator_seed", type=int, default=0, help="generatorのパラメータの初期値のシード")
parser.add_argument("--predictor_seed", type=int, default=0, help="predictorのパラメータの初期値のシード")
parser.add_argument("--batch_size", type=int, default=1, help="batchの大きさ")
parser.add_argument("--n_epochs", type=int, default=1000, help="学習させる回数")
# opt = parser.parse_args() # .pyの場合はこちらを使用(.ipynbの場合エラーになります)
opt = parser.parse_args(args=[]) # .ipynbの場合はこちらを使用

### NNモデルを作成($F$は学習済の$F$を読み込む)

In [7]:
preF_paramPath = path+"checkAR/parameters/p{0}/preLinearPredictor.pth".format(hat_p)
predictor.load_state_dict(torch.load(preF_paramPath))
predictor = models.LinearPredictNet(input_size=hat_p+1)
predictor = predictor.to(device)

torch.manual_seed(opt.generator_seed)
generator = models.LinearGenerator(input_size = hat_p+1)
generator = generator.to(device)

### $\varepsilon$からの重さを1にする

In [15]:
predictor.fc1.weight.data[0][0] = 1

In [16]:
predictor.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 1.0000, -0.4017,  0.7122, -0.5019,  1.0023]]))])

### 学習データとvalidationデータを分ける（9:1）

In [8]:
trainData = data[:N//10*9]
valData = data[N//10*9:]

### データローダの作成

In [9]:
import my_preprocess
train_dataset = my_preprocess.DataSet(tsData=trainData, hat_p=hat_p, device=device)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=opt.batch_size, shuffle=True, drop_last=True)
val_dataset = my_preprocess.DataSet(tsData=valData, hat_p=hat_p, device=device)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False, drop_last=True)

### 最適化手法の設定

In [10]:
optimizer_F = torch.optim.Adam(params=predictor.parameters(), lr=0.0001 )
# optimizer_F = torch.optim.SGD(params=predictor.parameters(), lr = 0.00005)

### Lossの設計

In [11]:
mseLoss = nn.MSELoss()
mseLoss = mseLoss.to(device)
# Variable(Wasserstein.pWasserstein(hat_normeps_t.view(opt.batch_size), p=1), requires_grad=True).to(device)

### 学習

In [17]:
start = time.time()
minLoss = np.inf
F_paramPath = path+"checkAR/parameters/p{0}/LinearPredictor.pth".format(hat_p)
G_paramPath = path+"checkAR/parameters/p{0}/LinearGenerator.pth".format(hat_p)

In [ ]:
for epoch in range(1, opt.n_epochs+1):# epochごとの処理
    